In [2]:
import pandas as pd 

In [3]:
df = pd.read_csv('clean_dataset.csv')

In [6]:
df.drop(columns='Unnamed: 0', inplace=True)

In [7]:
df

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,num_genres
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,...,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic j-pop singer-songwriter songwriter,4
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,...,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic chill,2
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,...,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic,1
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,...,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic,1
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,...,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89735,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,...,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music,1
89736,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,...,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music,1
89737,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,...,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music,1
89738,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,...,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music,1


In [11]:
#checking to make sure of unique 
df['track_id'].nunique() == df.shape[0]

True

In [17]:
ids = df['track_id'].to_list()

In [63]:
client_id = '1b0f76afafac4d4898811b950e1a0c08'
client_secret = '806820e9d6004f7c94723ef175ee547c'

In [106]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up Spotipy client
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [65]:
import time

def call_with_retry(func, max_retries=3, initial_delay=1, backoff_factor=2):
    retries = 0
    delay = initial_delay
    
    while retries < max_retries:
        try:
            result = func
            return result
        except spotipy.client.SpotifyException as e:
            if e.http_status == 429:
                print(f"Rate limited. Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= backoff_factor
                retries += 1
            else:
                raise e
    
    # Max retries reached
    print("Max retries reached. Giving up.")
    return None


In [72]:
from tqdm.notebook import tqdm

In [50]:
i = 0
for x in tqdm(range(100000)):
    i += 1
    continue

100%|██████████████████████████████| 100000/100000 [00:00<00:00, 2591460.05it/s]


In [69]:
# Define function to retrieve tracks in chunks of 50
def get_audio_features(ids):
    audio_features = []
    for i in tqdm(range(0, len(ids), 50)):
        chunk = ids[i:i+50]
        results = call_with_retry(sp.audio_features(chunk))
        audio_features += results
        print('Another chunk', i, end='')
    return audio_features


In [19]:
# Define function to retrieve tracks in chunks of 50
def get_tracks(ids):
    tracks = []
    for i in range(0, len(ids), 50):
        chunk = ids[i:i+50]
        results = call_with_retry(sp.tracks(chunk))
        tracks += results['tracks']
        print('Another chunk', i, end='')
    return tracks

# Define function to retrieve all tracks from a list of playlists
def get_all_tracks_from_playlists(playlists):
    track_ids = []
    for playlist in playlists:
        results = sp.playlist_tracks(playlist['id'])
        tracks = results['items']
        while results['next']:
            results = sp.next(results)
            tracks += results['items']
        for track in tracks:
            track_ids.append(track['track']['id'])
    all_tracks = get_tracks(track_ids)
    return all_tracks

In [27]:
# the pulling
pulled_jsons = get_tracks(ids)

Another chunk 0
Another chunk 50
Another chunk 100
Another chunk 150
Another chunk 200
Another chunk 250
Another chunk 300
Another chunk 350
Another chunk 400
Another chunk 450
Another chunk 500
Another chunk 550
Another chunk 600
Another chunk 650
Another chunk 700
Another chunk 750
Another chunk 800
Another chunk 850
Another chunk 900
Another chunk 950
Another chunk 1000
Another chunk 1050
Another chunk 1100
Another chunk 1150
Another chunk 1200
Another chunk 1250
Another chunk 1300
Another chunk 1350
Another chunk 1400
Another chunk 1450
Another chunk 1500
Another chunk 1550
Another chunk 1600
Another chunk 1650
Another chunk 1700
Another chunk 1750
Another chunk 1800
Another chunk 1850
Another chunk 1900
Another chunk 1950
Another chunk 2000
Another chunk 2050
Another chunk 2100
Another chunk 2150
Another chunk 2200
Another chunk 2250
Another chunk 2300
Another chunk 2350
Another chunk 2400
Another chunk 2450
Another chunk 2500
Another chunk 2550
Another chunk 2600
Another chunk 26

Another chunk 21050
Another chunk 21100
Another chunk 21150
Another chunk 21200
Another chunk 21250
Another chunk 21300
Another chunk 21350
Another chunk 21400
Another chunk 21450
Another chunk 21500
Another chunk 21550
Another chunk 21600
Another chunk 21650
Another chunk 21700
Another chunk 21750
Another chunk 21800
Another chunk 21850
Another chunk 21900
Another chunk 21950
Another chunk 22000
Another chunk 22050
Another chunk 22100
Another chunk 22150
Another chunk 22200
Another chunk 22250
Another chunk 22300
Another chunk 22350
Another chunk 22400
Another chunk 22450
Another chunk 22500
Another chunk 22550
Another chunk 22600
Another chunk 22650
Another chunk 22700
Another chunk 22750
Another chunk 22800
Another chunk 22850
Another chunk 22900
Another chunk 22950
Another chunk 23000
Another chunk 23050
Another chunk 23100
Another chunk 23150
Another chunk 23200
Another chunk 23250
Another chunk 23300
Another chunk 23350
Another chunk 23400
Another chunk 23450
Another chunk 23500


Another chunk 41600
Another chunk 41650
Another chunk 41700
Another chunk 41750
Another chunk 41800
Another chunk 41850
Another chunk 41900
Another chunk 41950
Another chunk 42000
Another chunk 42050
Another chunk 42100
Another chunk 42150
Another chunk 42200
Another chunk 42250
Another chunk 42300
Another chunk 42350
Another chunk 42400
Another chunk 42450
Another chunk 42500
Another chunk 42550
Another chunk 42600
Another chunk 42650
Another chunk 42700
Another chunk 42750
Another chunk 42800
Another chunk 42850
Another chunk 42900
Another chunk 42950
Another chunk 43000
Another chunk 43050
Another chunk 43100
Another chunk 43150
Another chunk 43200
Another chunk 43250
Another chunk 43300
Another chunk 43350
Another chunk 43400
Another chunk 43450
Another chunk 43500
Another chunk 43550
Another chunk 43600
Another chunk 43650
Another chunk 43700
Another chunk 43750
Another chunk 43800
Another chunk 43850
Another chunk 43900
Another chunk 43950
Another chunk 44000
Another chunk 44050


Another chunk 62100
Another chunk 62150
Another chunk 62200
Another chunk 62250
Another chunk 62300
Another chunk 62350
Another chunk 62400
Another chunk 62450
Another chunk 62500
Another chunk 62550
Another chunk 62600
Another chunk 62650
Another chunk 62700
Another chunk 62750
Another chunk 62800
Another chunk 62850
Another chunk 62900
Another chunk 62950
Another chunk 63000
Another chunk 63050
Another chunk 63100
Another chunk 63150
Another chunk 63200
Another chunk 63250
Another chunk 63300
Another chunk 63350
Another chunk 63400
Another chunk 63450
Another chunk 63500
Another chunk 63550
Another chunk 63600
Another chunk 63650
Another chunk 63700
Another chunk 63750
Another chunk 63800
Another chunk 63850
Another chunk 63900
Another chunk 63950
Another chunk 64000
Another chunk 64050
Another chunk 64100
Another chunk 64150
Another chunk 64200
Another chunk 64250
Another chunk 64300
Another chunk 64350
Another chunk 64400
Another chunk 64450
Another chunk 64500
Another chunk 64550


Another chunk 82600
Another chunk 82650
Another chunk 82700
Another chunk 82750
Another chunk 82800
Another chunk 82850
Another chunk 82900
Another chunk 82950
Another chunk 83000
Another chunk 83050
Another chunk 83100
Another chunk 83150
Another chunk 83200
Another chunk 83250
Another chunk 83300
Another chunk 83350
Another chunk 83400
Another chunk 83450
Another chunk 83500
Another chunk 83550
Another chunk 83600
Another chunk 83650
Another chunk 83700
Another chunk 83750
Another chunk 83800
Another chunk 83850
Another chunk 83900
Another chunk 83950
Another chunk 84000
Another chunk 84050
Another chunk 84100
Another chunk 84150
Another chunk 84200
Another chunk 84250
Another chunk 84300
Another chunk 84350
Another chunk 84400
Another chunk 84450
Another chunk 84500
Another chunk 84550
Another chunk 84600
Another chunk 84650
Another chunk 84700
Another chunk 84750
Another chunk 84800
Another chunk 84850
Another chunk 84900
Another chunk 84950
Another chunk 85000
Another chunk 85050


In [73]:
pulled_features = get_audio_features(ids)

  0%|          | 0/1795 [00:00<?, ?it/s]

Another chunk 0Another chunk 50Another chunk 100Another chunk 150Another chunk 200Another chunk 250Another chunk 300Another chunk 350Another chunk 400Another chunk 450Another chunk 500Another chunk 550Another chunk 600Another chunk 650Another chunk 700Another chunk 750Another chunk 800Another chunk 850Another chunk 900Another chunk 950Another chunk 1000Another chunk 1050Another chunk 1100Another chunk 1150Another chunk 1200Another chunk 1250Another chunk 1300Another chunk 1350Another chunk 1400Another chunk 1450Another chunk 1500Another chunk 1550Another chunk 1600Another chunk 1650Another chunk 1700Another chunk 1750Another chunk 1800Another chunk 1850Another chunk 1900Another chunk 1950Another chunk 2000Another chunk 2050Another chunk 2100Another chunk 2150Another chunk 2200Another chunk 2250Another chunk 2300Another chunk 2350Another chunk 2400Another chunk 2450Another chunk 2500Another chunk 2550Another chunk 2600Another chunk 2650Another chunk 2700Another chunk 2750Another chunk 2

Another chunk 22200Another chunk 22250Another chunk 22300Another chunk 22350Another chunk 22400Another chunk 22450Another chunk 22500Another chunk 22550Another chunk 22600Another chunk 22650Another chunk 22700Another chunk 22750Another chunk 22800Another chunk 22850Another chunk 22900Another chunk 22950Another chunk 23000Another chunk 23050Another chunk 23100Another chunk 23150Another chunk 23200Another chunk 23250Another chunk 23300Another chunk 23350Another chunk 23400Another chunk 23450Another chunk 23500Another chunk 23550Another chunk 23600Another chunk 23650Another chunk 23700Another chunk 23750Another chunk 23800Another chunk 23850Another chunk 23900Another chunk 23950Another chunk 24000Another chunk 24050Another chunk 24100Another chunk 24150Another chunk 24200Another chunk 24250Another chunk 24300Another chunk 24350Another chunk 24400Another chunk 24450Another chunk 24500Another chunk 24550Another chunk 24600Another chunk 24650Another chunk 24700Another chunk 24750Another chun

Another chunk 43800Another chunk 43850Another chunk 43900Another chunk 43950Another chunk 44000Another chunk 44050Another chunk 44100Another chunk 44150Another chunk 44200Another chunk 44250Another chunk 44300Another chunk 44350Another chunk 44400Another chunk 44450Another chunk 44500Another chunk 44550Another chunk 44600Another chunk 44650Another chunk 44700Another chunk 44750Another chunk 44800Another chunk 44850Another chunk 44900Another chunk 44950Another chunk 45000Another chunk 45050Another chunk 45100Another chunk 45150Another chunk 45200Another chunk 45250Another chunk 45300Another chunk 45350Another chunk 45400Another chunk 45450Another chunk 45500Another chunk 45550Another chunk 45600Another chunk 45650Another chunk 45700Another chunk 45750Another chunk 45800Another chunk 45850Another chunk 45900Another chunk 45950Another chunk 46000Another chunk 46050Another chunk 46100Another chunk 46150Another chunk 46200Another chunk 46250Another chunk 46300Another chunk 46350Another chun

Another chunk 65400Another chunk 65450Another chunk 65500Another chunk 65550Another chunk 65600Another chunk 65650Another chunk 65700Another chunk 65750Another chunk 65800Another chunk 65850Another chunk 65900Another chunk 65950Another chunk 66000Another chunk 66050Another chunk 66100Another chunk 66150Another chunk 66200Another chunk 66250Another chunk 66300Another chunk 66350Another chunk 66400Another chunk 66450Another chunk 66500Another chunk 66550Another chunk 66600Another chunk 66650Another chunk 66700Another chunk 66750Another chunk 66800Another chunk 66850Another chunk 66900Another chunk 66950Another chunk 67000Another chunk 67050Another chunk 67100Another chunk 67150Another chunk 67200Another chunk 67250Another chunk 67300Another chunk 67350Another chunk 67400Another chunk 67450Another chunk 67500Another chunk 67550Another chunk 67600Another chunk 67650Another chunk 67700Another chunk 67750Another chunk 67800Another chunk 67850Another chunk 67900Another chunk 67950Another chun

Another chunk 87000Another chunk 87050Another chunk 87100Another chunk 87150Another chunk 87200Another chunk 87250Another chunk 87300Another chunk 87350Another chunk 87400Another chunk 87450Another chunk 87500Another chunk 87550Another chunk 87600Another chunk 87650Another chunk 87700Another chunk 87750Another chunk 87800Another chunk 87850Another chunk 87900Another chunk 87950Another chunk 88000Another chunk 88050Another chunk 88100Another chunk 88150Another chunk 88200Another chunk 88250Another chunk 88300Another chunk 88350Another chunk 88400Another chunk 88450Another chunk 88500Another chunk 88550Another chunk 88600Another chunk 88650Another chunk 88700Another chunk 88750Another chunk 88800Another chunk 88850Another chunk 88900Another chunk 88950Another chunk 89000Another chunk 89050Another chunk 89100Another chunk 89150Another chunk 89200Another chunk 89250Another chunk 89300Another chunk 89350Another chunk 89400Another chunk 89450Another chunk 89500Another chunk 89550Another chun

In [31]:
len(pulled_jsons) == df.shape[0]

True

In [32]:
type(pulled_jsons

list

In [34]:
import pickle

In [46]:
# saving as pickle
#with open("pulled_songs.pkl", "wb") as f:
#    pickle.dump(pulled_jsons, f)

In [41]:
# opening pickle
with open('pulled_songs.pkl', 'rb') as f:
    new_pulled = pickle.load(f)

In [74]:
#with open("pulled_features_.pkl", "wb") as f:
#    pickle.dump(pulled_features, f)

In [55]:
#confiming the pulled file is exactly what my data set is.
for x in tqdm(range(df.shape[0])):


    if new_pulled[x]['id'] == ids[x]:
        continue
    else:
        print('fuck')
        break

100%|████████████████████████████████| 89740/89740 [00:00<00:00, 1068820.34it/s]


In [75]:
len(pulled_features)

89740

In [81]:
#checking if features match deatils
for x in tqdm(range(df.shape[0])):
    if pulled_features[x]['id'] == new_pulled[x]['id']:
        continue
    else:
        print('fuck')
        break

  0%|          | 0/89740 [00:00<?, ?it/s]

In [56]:
new_pulled[0]

{'album': {'album_group': 'single',
  'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1S2S00lgLYLGHWA44qGEUs'},
    'href': 'https://api.spotify.com/v1/artists/1S2S00lgLYLGHWA44qGEUs',
    'id': '1S2S00lgLYLGHWA44qGEUs',
    'name': 'Gen Hoshino',
    'type': 'artist',
    'uri': 'spotify:artist:1S2S00lgLYLGHWA44qGEUs'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BF',
   'BG',
   'BH',
   'BI',
   'BJ',
   'BN',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BY',
   'BZ',
   'CA',
   'CD',
   'CG',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',
   'DE',
   'DJ',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'ET',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GA',
   'GB',
   'GD',
   'GE',
   'GH',
   'GM',
   'GN',
   'GQ',
   'GR',
   'GT',
   'GW',
   'GY',

In [182]:
# define search query
query = "artist:J Cole"

# use 'search' method to find tracks matching query
results = sp.search(q=query, type="track")

# print the track information for each result
for track in results["tracks"]["items"]:
    print(f"{track['name']} by {track['artists'][0]['name']}")

No Role Modelz by J. Cole
on the street (with J. Cole) by j-hope
Wet Dreamz by J. Cole
MIDDLE CHILD by J. Cole
Work Out by J. Cole
m y . l i f e (with 21 Savage & Morray) by J. Cole
Power Trip (feat. Miguel) by J. Cole
Love Yourz by J. Cole
She Knows (feat. Amber Coffman & Cults) by J. Cole
Kevin’s Heart by J. Cole


In [183]:
results['tracks']['items'][0]['id']

'68Dni7IE4VyPkTOH9mRWHr'

In [110]:
sp.recommendations(seed_artists=['Drake'],min_popularity=50,limit=10)

HTTP Error for GET to https://api.spotify.com/v1/recommendations with Params: {'limit': 10, 'seed_artists': 'Drake', 'min_popularity': 50} returned 400 due to invalid request


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/recommendations?limit=10&seed_artists=Drake&min_popularity=50:
 invalid request, reason: None

In [194]:
seed_track =[ "3F5CgOj3wFlRv51JsHbxhe",'4Izx6tlGNTpflkxmVLQqkq','5Qy6a5KzM4XlRxsNcGYhgH','2HbKqm4o0w5wEeEFXm2sD4','68Dni7IE4VyPkTOH9mRWHr']

# set parameters for recommendation algorithm
params = {
    "limit": 10, # number of recommendations to return
    "seed_tracks": seed_track, # list of seed tracks to base recommendations on
    "min_popularity": 50, # minimum popularity score for recommended tracks
    'max_popularity': 60,
    "seed_genre": ['hip-hop']
}

# use 'recommendations' method to get recommended tracks
results = sp.recommendations(**params)

In [196]:
results['tracks'][6]['name']#['name']

'Ferrari'

In [127]:
results['tracks'][0].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [128]:
results = sp.recommendation_genre_seeds()

# print the available genre seeds
print(results['genres'])


['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-po

In [130]:
len(results['genres'])

126